In [1]:
import pandas as pd
import os
from pathlib import Path

DEFAULT_TRADE_TIME = "17:00:00+00:00" # noon UTC

DATA_DIR = Path(os.getcwd()).parent.parent.parent / "data"
RAW_DIR = DATA_DIR / "trades" / "raw" 
OUTPUT_DIR = DATA_DIR / "trades" / "clean"

INPUT_EXPORTED_FILE = RAW_DIR / "vanguard_raw_exported.csv"
INPUT_MANUAL_TXS_FILE = RAW_DIR /  "vanguard_raw_manual_transactions.csv"
INPUT_MANUAL_PRICES_FILE = RAW_DIR / "vanguard_raw_manual_prices.csv"

OUTPUT_FILE = OUTPUT_DIR / "vanguard_clean.csv"


In [2]:
exported_raw_df = pd.read_csv(INPUT_EXPORTED_FILE)
manual_transactions_raw_df = pd.read_csv(INPUT_MANUAL_TXS_FILE)
manual_prices_raw_df = pd.read_csv(INPUT_MANUAL_PRICES_FILE)

### Dataframe Schema Information

#### Vanguard Exported
- **Columns**: ['Account Number', 'Trade Date', 'Settlement Date', 'Transaction Type', 'Transaction Description', 'Investment Name', 'Symbol', 'Shares', 'Share Price', 'Principal Amount', 'Commissions and Fees', 'Net Amount', 'Accrued Interest', 'Account Type']

#### Manual Transaction Data
- **Columns**: ['Subcategory', 'Total World', 'Large Cap', 'Mid Cap', 'Small Cap', 'International', 'Emerging Markets', 'Inflation Protected', 'Tax Exempt', 'International.1', 'Gold', 'Real Estate']
- **Structure**: 
  - Row 0: Asset tickers (VT, VOO, VO, VB, VXUS, VWO, VTIP, VTEB, BNDX, AAAU, VNQ)
  - Row 1: Subcategory totals
  - Rows 2+: Date-indexed transaction data with share quantities

#### Manual Price Data
- **Columns**: ['Category', 'Stocks', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Bonds', 'Unnamed: 8', 'Unnamed: 9', 'Gold', 'Real Estate']
- **Structure**:
  - Row 0: Category totals
  - Row 1: Subcategory labels
  - Row 2: Asset tickers (VT, VOO, VO, VB, VXUS, VWO, VTIP, VTEB, BNDX, AAAU, VNQ)
  - Row 3: Subcategory totals
  - Rows 4+: Date-indexed price data with dollar amounts


### Last 18 Months - Use Exported Data

In [3]:
exported_data = []

for _, row in exported_raw_df.iterrows():
    if row["Transaction Type"] == "Buy" and not pd.isna(row["Shares"]) and not pd.isna(row["Share Price"]):
        exported_data.append({
            "date": f"{row['Settlement Date']}",
            "asset": row["Symbol"],
            "price": row["Share Price"],
            "quantity": row["Shares"]
        })

exported_df = pd.DataFrame(exported_data)
exported_df = exported_df.sort_values(["date", "asset"]).reset_index(drop=True)
exported_df.head()

,date,asset,price,quantity
0,2024-02-12,AAAU,20.1182,18.0
1,2024-02-12,AAAU,20.1199,6.0
2,2024-02-12,VB,211.2550,8.0
3,2024-02-12,VB,211.2500,1.0
4,2024-02-12,VO,233.2650,4.0


## Earlier Data - Use Manual Data from Gsheets

In [4]:
# Extract tickers from the asset row (index 0 in transactions, index 2 in prices)
tx_tickers = manual_transactions_raw_df.iloc[0, 1:].values  # Skip first column
price_tickers = manual_prices_raw_df.iloc[2, 1:].values    # Skip first column

# Get the data starting from the actual transaction/price rows
tx_data = manual_transactions_raw_df.iloc[2:].copy()  # Start from index 2
price_data = manual_prices_raw_df.iloc[4:].copy()     # Start from index 4

tx_data = tx_data.reset_index(drop=True)
price_data = price_data.reset_index(drop=True)

In [5]:
# Create the combined data
manual_data = []
for i in range(len(tx_data)):
    date = tx_data.iloc[i, 0]  # First column is the date
    
    # Iterate through each ticker column
    for j, ticker in enumerate(tx_tickers):
        if pd.isna(ticker):  # Skip NaN tickers
            continue
            
        # Get quantity from transactions (j+1 because we skip the date column)
        quantity = tx_data.iloc[i, j+1]
        
        # Get price from prices dataframe (j+1 because we skip the date column)
        price_str = price_data.iloc[i, j+1] if i < len(price_data) else None
        
        # Only add row if both quantity and price exist and are not NaN
        if not pd.isna(quantity) and not pd.isna(price_str) and quantity != 0:
            # Clean price string (remove $ and commas)
            if isinstance(price_str, str):
                price = float(price_str.replace("$", "").replace(",", ""))
            else:
                price = float(price_str) if not pd.isna(price_str) else None # type: ignore
                
            if price is not None:
                manual_data.append({
                    "date": date,
                    "asset": ticker,
                    "price": price,
                    "quantity": quantity
                })

# Create the final dataframe
manual_df = pd.DataFrame(manual_data)
manual_df["date"] = manual_df["date"].map(lambda i: pd.to_datetime(i).strftime('%Y-%m-%d'))

cutoff_date = exported_raw_df[exported_raw_df["Transaction Type"] == "Buy"]["Trade Date"].min()
manual_df = manual_df[manual_df.date < cutoff_date]

manual_df = manual_df.sort_values(["date", "asset"]).reset_index(drop=True)
manual_df.head()

,date,asset,price,quantity
0,2021-06-03,VB,223.60,1
1,2021-06-03,VOO,384.13,1
2,2021-06-03,VT,102.79,6
3,2021-06-03,VWO,54.61,1
4,2021-06-18,AAAU,17.65,5


### Final Data

In [6]:
vanguard_df = pd.concat([exported_df, manual_df], ignore_index=True)
vanguard_df = vanguard_df.sort_values(["date", "asset"]).reset_index(drop=True)

vanguard_df["id"] = vanguard_df.index.map(lambda i: f"vanguard-{i}")
vanguard_df["platform"] = "vanguard"
vanguard_df["fees"] = 0
vanguard_df["action"] = "BUY"
vanguard_df["cost"] = (vanguard_df["quantity"].astype(float) * vanguard_df["price"].astype(float))
vanguard_df["cost"] = vanguard_df["cost"].map(lambda i: str(round(i, 2)))
vanguard_df["value"] = vanguard_df["cost"]

vanguard_df = vanguard_df[["id", "platform", "date", "action", "asset", "price", "quantity", "fees", "cost", "value"]]
vanguard_df.head()

,id,platform,date,action,asset,price,quantity,fees,cost,value
0,vanguard-0,vanguard,2021-06-03,BUY,VB,223.60,1,0,223.6,223.6
1,vanguard-1,vanguard,2021-06-03,BUY,VOO,384.13,1,0,384.13,384.13
2,vanguard-2,vanguard,2021-06-03,BUY,VT,102.79,6,0,616.74,616.74
3,vanguard-3,vanguard,2021-06-03,BUY,VWO,54.61,1,0,54.61,54.61
4,vanguard-4,vanguard,2021-06-18,BUY,AAAU,17.65,5,0,88.25,88.25


In [7]:
assert vanguard_df.shape == (491, 10)

In [8]:
vanguard_df.to_csv(OUTPUT_FILE, index=False)